# Messing Around with Scikit-Learn

## The basics: What is scikit-learn?

Scikit-learn, or `sklearn`, is a machine learning package for python. It is built on the familiar scientific python tools (numpy, scipy, matplotlib, etc.) and thus plays very nicely with data in the form of numpy arrays (a cursory googling shows people are also working on [better integrating sklearn with other popular pythonic data structures like pandas DataFrames](https://github.com/paulgb/sklearn-pandas)).

This notebook is really just an aggregation of some of the things from the sklearn documentation that I thought were interesting and/or useful. The [documentation](http://scikit-learn.org/stable/documentation.html) is really quite well organized in my opinion, so if you are curious about the layout of scikit learn or whether it has specific capabilities that you're interested in, go check it out!

## When should you use scikit-learn?

When you have data that consist of multiple samples and you'd like to find patterns within that data to predict properties of unknown data. The [documentation](http://scikit-learn.org/stable/tutorial/basic/tutorial.html) breaks down learning problems into the following categories:

 - **Supervised learning**
   
   Where you have input data and corresponding properties (labels, for example) that you'd like to learn from 